# Heart Attack Risk Prediciton

In [1]:
#import library
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os
import pandas as pd

## Clean Dataset

Menghapus kolom yang tidak dibutuhkan dalam model

In [2]:
df = pd.read_csv('dataset/heart_attack_dataset.csv')

if 'Patient ID' in df.columns:
    df.drop(columns=['Patient ID', 'Blood Pressure', 'Country', 'Continent', 'Hemisphere', 'Diet'], inplace=True)

df.head()

,Age,Sex,Cholesterol,Heart Rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,Exercise Hours Per Week,Previous Heart Problems,Medication Use,Stress Level,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Heart Attack Risk
0,67,Male,208,72,0,0,1,0,0,4.168189,0,0,9,6.615001,261404,31.251233,286,0,6,0
1,21,Male,389,98,1,1,1,1,1,1.813242,1,0,1,4.963459,285768,27.194973,235,1,7,0
2,21,Female,324,72,1,0,0,0,0,2.078353,1,1,9,9.463426,235282,28.176571,587,4,4,0
3,84,Male,383,73,1,1,1,0,1,9.828130,1,0,9,7.648981,125640,36.464704,378,3,4,0
4,66,Male,318,93,1,1,1,1,0,5.804299,1,0,6,1.514821,160555,21.809144,231,1,5,0


In [3]:
df.to_csv('clean_dataset/heart_attack_clean.csv', index=False)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8763 entries, 0 to 8762
Data columns (total 20 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Age                              8763 non-null   int64  
 1   Sex                              8763 non-null   object 
 2   Cholesterol                      8763 non-null   int64  
 3   Heart Rate                       8763 non-null   int64  
 4   Diabetes                         8763 non-null   int64  
 5   Family History                   8763 non-null   int64  
 6   Smoking                          8763 non-null   int64  
 7   Obesity                          8763 non-null   int64  
 8   Alcohol Consumption              8763 non-null   int64  
 9   Exercise Hours Per Week          8763 non-null   float64
 10  Previous Heart Problems          8763 non-null   int64  
 11  Medication Use                   8763 non-null   int64  
 12  Stress Level        

## Set Variable

In [5]:
PIPELINE_NAME = "hanhanhanny-pipeline"
SCHEMA_PIPELINE_NAME = "heart-attack-schema"

# Directory to store pipeline artifacts
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

# Path to a SQLite DB file for MLMD storage
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Directory to export created models
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

In [6]:
# Initialize Interactive Context
DATA_ROOT = "clean_dataset"  # the data is placed in the "clean_dataset" directory
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

## Data Ingestion

Menggunakan ExampleGen untuk membagi dataset sebelum proses training dan evaluasi dengan rasio 8:2

In [7]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [8]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

## Data Validation

Membuat StatisticGen yang berfungsi unntuk membuat statitik dari seluruh feature yang ada

In [9]:
statistics_gen = StatisticsGen(
    examples = example_gen.outputs["examples"]
)
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [10]:
interactive_context.show(statistics_gen.outputs["statistics"])

Membuat SchemaGen untuk memperoleh informasi dari feature dan label

In [11]:
schema_gen = SchemaGen(    statistics=statistics_gen.outputs["statistics"]
)
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [12]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'Age',INT,required,,-
'Alcohol Consumption',INT,required,,-
'BMI',FLOAT,required,,-
'Cholesterol',INT,required,,-
'Diabetes',INT,required,,-
'Exercise Hours Per Week',FLOAT,required,,-
'Family History',INT,required,,-
'Heart Attack Risk',INT,required,,-
'Heart Rate',INT,required,,-


,Values
Domain,
'Sex',"'Female', 'Male'"


Dari SchemaGen yang telah dibuat, terdapat 20 feature yang akan digunakan untuk menentukan prediksi resiko serangan jantung. Terdapat berbagai tipe data yaitu int, float, dan string.

In [13]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs["statistics"],
    schema=schema_gen.outputs["schema"]
)

interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

ExampleValidator digunakan untuk mendeteksi anomali pada data, menggunakan StatisticGen dan SchemaGen yang telah dibuat sebelumnya

In [14]:
interactive_context.show(example_validator.outputs["anomalies"])

Dapat dilihat bahwa tidak ditemukan anomali, sehingga data akan siap digunakan untuk proses pipeline selanjutnya.

## Data Preprocessing

Membuat heart_attack_transform.py untuk menyimpan proses transform pada module

In [15]:
TRANSFORM_MODULE_FILE = "heart_attack_transform.py"

In [16]:
%%writefile {TRANSFORM_MODULE_FILE}

import tensorflow as tf
import tensorflow_transform as tft
import os

# Set a custom temporary directory
os.environ['TF_TFT_TMP_DIR'] = '/path/to/your/temp/dir'

def transformed_key(key):
    """Renaming transformed features"""
    return key + "_xf"


def preprocessing_fn(inputs):
    """
    Preprocess input features into transformed features
    
    Args:
        inputs: map from feature keys to raw features.
    
    Return:
        outputs: map from feature keys to transformed features.    

    Description:
        - apply one hot encoding to categorical features
        - apply standardization to float features and int features that are not binary
        - apply renaming of transformed features except for one hot encoded features
    """
    
    outputs = {}

    # Convert "Female" and "Male" to 1 and 0
    gender_lookup = tf.lookup.StaticHashTable(
        initializer=tf.lookup.KeyValueTensorInitializer(
            keys=tf.constant(['Female', 'Male']),
            values=tf.constant([1, 0]),
        ),
        default_value=tf.constant(-1),
    )
    outputs[transformed_key("Sex")] = tf.cast(gender_lookup.lookup(inputs["Sex"]), tf.int64) 

    # Standardize numerical features
    outputs[transformed_key("Cholesterol")] = tft.scale_to_0_1(inputs["Cholesterol"])
    outputs[transformed_key("Sedentary_Hours_Per_Day")] = tft.scale_to_0_1(inputs["Sedentary Hours Per Day"])
    outputs[transformed_key("BMI")] = tft.scale_to_0_1(inputs["BMI"])
    outputs[transformed_key("Exercise_Hours_Per_Week")] = tft.scale_to_0_1(inputs["Exercise Hours Per Week"])


    # Binary features (no transformation)
    outputs["Age"] = inputs["Age"]
    outputs["Heart_Rate"] = inputs["Heart Rate"]
    outputs["Triglycerides"] = inputs["Triglycerides"]
    outputs["Stress_Level"] = inputs["Stress Level"]
    outputs["Physical_Activity_Days_Per_Week"] = inputs["Physical Activity Days Per Week"]
    outputs["Sleep_Hours_Per_Day"] = inputs["Sleep Hours Per Day"]
    outputs["Smoking"] = inputs["Smoking"]
    outputs["Diabetes"] = inputs["Diabetes"]
    outputs["Family_History"] = inputs["Family History"]
    outputs["Obesity"] = inputs["Obesity"]
    outputs["Income"] = inputs["Income"]
    outputs["Alcohol_Consumption"] = inputs["Alcohol Consumption"]
    outputs["Previous_Heart_Problems"] = inputs["Previous Heart Problems"]
    outputs["Medication_Use"] = inputs["Medication Use"]

    # Target feature
    outputs[transformed_key("Heart Attack Risk")] = tf.cast(inputs["Heart Attack Risk"], tf.int64)

    return outputs

Overwriting heart_attack_transform.py


Module heart_attack_transform.py digunakan untuk mengubah kolom 'Sex' yang mulanya string, menjadi int dengan mengubah Female dan Male menjadi 1 dan 0. Selain itu, melakukan standarisasi pada kolom yang mengandung data numerical.

Kemudian menjalankan file transform bersamaan dengan ExampleGen dan SchemaGen.

In [17]:
transform = Transform(
    examples=example_gen.outputs["examples"],
    schema=schema_gen.outputs["schema"],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: pipelines\hanhanhanny-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\71421146c0454c458cbec1f9e1711e42\assets


INFO:tensorflow:Assets written to: pipelines\hanhanhanny-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\71421146c0454c458cbec1f9e1711e42\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

## Model Training

Membuat file module heart_attack_trainer.py untuk menyimpan proses training dan tuning pada model

In [18]:
TRAINER_TUNER_MODULE_FILE = "heart_attack_trainer.py"

In [19]:
%%writefile {TRAINER_TUNER_MODULE_FILE}

import tensorflow as tf
import tensorflow_transform as tft 
from tensorflow.keras import layers
import os
from keras_tuner.engine import base_tuner
from keras_tuner import RandomSearch, HyperParameters
from tfx.components.trainer.fn_args_utils import FnArgs
from typing import NamedTuple, Dict, Text, Any

LABEL_KEY = "Heart Attack Risk"

def transformed_key(key):
    """Renaming transformed features"""
    return key + "_xf"

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def get_hyperparameters() -> HyperParameters:
    """Returns hyperparameters for building model"""
    hp = HyperParameters()
    hp.Int('units', min_value=32, max_value=512, step=32, default=128)
    hp.Int('num_layers', min_value=1, max_value=4, step=1, default=3)
    hp.Float('learning_rate', min_value=1e-4, max_value=1e-1, sampling='LOG', default=1e-2)
    hp.Float('dropout_rate', min_value=0.0, max_value=0.5, step=0.1, default=0.2)
    return hp

def input_fn(file_pattern, 
             tf_transform_output,
             num_epochs=None,
             batch_size=64)->tf.data.Dataset:
    """Get post_transform feature & create batches of data"""
    
    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())
    
    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_key(LABEL_KEY))
    dataset = dataset.shuffle(buffer_size=10000)
    return dataset

def model_builder(hparams: HyperParameters):
    """Build machine learning model"""
    inputs = {
        transformed_key('Cholesterol'): tf.keras.Input(shape=(1,), name=transformed_key('Cholesterol'), dtype=tf.float32), 
        transformed_key('Sedentary_Hours_Per_Day'): tf.keras.Input(shape=(1,), name=transformed_key('Sedentary_Hours_Per_Day'), dtype=tf.float32),
        transformed_key('BMI'): tf.keras.Input(shape=(1,), name=transformed_key('BMI'), dtype=tf.float32),
        transformed_key('Exercise_Hours_Per_Week'): tf.keras.Input(shape=(1,), name=transformed_key('Exercise_Hours_Per_Week'), dtype=tf.float32),
        transformed_key('Sex'): tf.keras.Input(shape=(1,), name=transformed_key('Sex'), dtype=tf.float32), 
        'Age': tf.keras.Input(shape=(1,), name='Age', dtype=tf.float32), 
        'Income': tf.keras.Input(shape=(1,), name='Income', dtype=tf.float32), 
        'Heart_Rate': tf.keras.Input(shape=(1,), name='Heart_Rate', dtype=tf.float32), 
        'Smoking': tf.keras.Input(shape=(1,), name='Smoking', dtype=tf.float32), 
        'Stress_Level': tf.keras.Input(shape=(1,), name='Stress_Level', dtype=tf.float32), 
        'Triglycerides': tf.keras.Input(shape=(1,), name='Triglycerides', dtype=tf.float32), 
        'Diabetes': tf.keras.Input(shape=(1,), name='Diabetes', dtype=tf.float32), 
        'Sleep_Hours_Per_Day': tf.keras.Input(shape=(1,), name='Sleep_Hours_Per_Day', dtype=tf.float32),
        'Physical_Activity_Days_Per_Week': tf.keras.Input(shape=(1,), name='Physical_Activity_Days_Per_Week', dtype=tf.float32),
        'Family_History': tf.keras.Input(shape=(1,), name='Family_History', dtype=tf.float32),
        'Obesity': tf.keras.Input(shape=(1,), name='Obesity', dtype=tf.float32), 
        'Alcohol_Consumption': tf.keras.Input(shape=(1,), name='Alcohol_Consumption', dtype=tf.float32), 
        'Previous_Heart_Problems': tf.keras.Input(shape=(1,), name='Previous_Heart_Problems', dtype=tf.float32), 
        'Medication_Use': tf.keras.Input(shape=(1,), name='Medication_Use', dtype=tf.float32) 
    }
    
    # Combine all inputs into a single tensor
    concatenated_inputs = layers.Concatenate()(list(inputs.values()))

    x = layers.Dense(hparams.get('units'), activation='relu')(concatenated_inputs)
    for _ in range(hparams.get('num_layers') - 1):
        x = layers.Dense(hparams.get('units') // 2, activation='relu')(x)
        x = layers.Dropout(hparams.get('dropout_rate'))(x)
    x = layers.BatchNormalization()(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        loss='binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(learning_rate=hparams.get('learning_rate')),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )
    
    model.summary()
    return model

# Tuner component will run this function
TunerFnResult = NamedTuple('TunerFnResult', [('tuner', RandomSearch),
                                             ('fit_kwargs', Dict[Text, Any])])

def tuner_fn(fn_args: FnArgs) -> TunerFnResult:
    """
    Build the tuner using the KerasTuner API.
    Args:
        fn_args: Holds args as name/value pairs.
        - working_dir: working dir for tuning.
        - train_files: List of file paths containing training tf.Example data.
        - eval_files: List of file paths containing eval tf.Example data.
        - train_steps: number of train steps.
        - eval_steps: number of eval steps.
        - schema_path: optional schema of the input data.
        - transform_graph_path: optional transform graph produced by TFT.
    Returns:
        A namedtuple contains the following:
        - tuner: A RandomSearch tuner that will be used for tuning.
        - fit_kwargs: Args to pass to tuner's run_trial function for fitting the
                        model, e.g., the training and validation dataset. Required
                        args depend on the above tuner's implementation.
    """
    hp = get_hyperparameters()
    # Define tuner
    tuner = RandomSearch(
        model_builder,
        objective='val_binary_accuracy',
        max_trials=20,
        directory=fn_args.working_dir,
        project_name='heart_attack_risk_classification',
        hyperparameters=hp
    )

    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    eval_set = input_fn(fn_args.eval_files, tf_transform_output, 10)

    return TunerFnResult(
        tuner=tuner,
        fit_kwargs={
            'x': train_set,
            'validation_data': eval_set
        }
    )

def _get_serve_tf_examples_fn(model, tf_transform_output):
    model.tft_layer = tf_transform_output.transform_features_layer()
    
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)
        return model(transformed_features)
        
    return serve_tf_examples_fn

def run_fn(fn_args: FnArgs) -> None:
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir, update_freq='batch'
    )
    
    es = tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', mode='max', verbose=1, patience=10)
    mc = tf.keras.callbacks.ModelCheckpoint(fn_args.serving_model_dir, monitor='val_binary_accuracy', mode='max', verbose=1, save_best_only=True)
    
    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    # Create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, num_epochs=10)
    eval_set = input_fn(fn_args.eval_files, tf_transform_output, num_epochs=10)

    hp = get_hyperparameters()
    model = model_builder(hp)
    
    # Train the model
    model.fit(
        x=train_set,
        validation_data=eval_set,
        callbacks=[tensorboard_callback, es, mc],
        steps_per_epoch=100, 
        validation_steps=100,
        epochs=10
    )
    
    signatures = {
        'serving_default': _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
            tf.TensorSpec(
                shape=[None],
                dtype=tf.string,
                name='examples'))
    }
    
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)


Overwriting heart_attack_trainer.py


In [20]:
from tfx.components import Tuner
from tfx.proto import trainer_pb2
 
tuner = Tuner(
    module_file=os.path.abspath(TRAINER_TUNER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=500),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=100)
    )
 
interactive_context.run(tuner)

Trial 20 Complete [00h 00m 13s]
val_binary_accuracy: 0.6629866361618042

Best val_binary_accuracy So Far: 0.6629866361618042
Total elapsed time: 00h 03m 59s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Results summary
Results in pipelines\hanhanhanny-pipeline\.temp\6\heart_attack_risk_classification
Showing 10 best trials
Objective(name="val_binary_accuracy", direction="max")

Trial 00 summary
Hyperparameters:
units: 192
num_layers: 4
learning_rate: 0.0008446639834657536
dropout_rate: 0.2
Score: 0.6629866361618042

Trial 01 summary
Hyperparameters:
units: 416
num_layers: 2
learning_rate: 0.0006936298725511624
dropout_rate: 0.0
Score: 0.6629866361618042

Trial 02 summary
Hyperparameters:
units: 64
num_layers: 1
learning_rate: 0.0010563102652253482
dropout_rate: 0.1
Score: 0.6629866361618042

Trial 03 summary
Hyperparameters:
units: 160
num_layers: 4
learning_rate: 0.017563360177390987
dropout_rate: 0.30000000000000004
Score: 0.6629866361618042

Trial 04 summary
Hyperparameters:
units: 64
num_layers: 4
learning_rate: 0.0007846586819444597
dropout_rate: 0.30000000000000004
Score: 0.6629866361618042

Trial 05 summary
Hyperparameters:
units: 320
num_layers: 1
learning_rate: 0.000654279881

ExecutionResult(
    component_id: Tuner
    execution_id: 6
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={})
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}))

In [21]:
from tfx.proto import trainer_pb2
 
trainer = Trainer(
    module_file = os.path.abspath(TRAINER_TUNER_MODULE_FILE),
    examples = transform.outputs['transformed_examples'],
    transform_graph = transform.outputs['transform_graph'],
    schema = schema_gen.outputs['schema'],
    hyperparameters = tuner.outputs['best_hyperparameters'],
    train_args = trainer_pb2.TrainArgs(splits=['train']),
    eval_args = trainer_pb2.EvalArgs(splits=['eval'])
)

interactive_context.run(trainer)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Cholesterol_xf (InputLayer)    [(None, 1)]          0           []                               
                                                                                                  
 Sedentary_Hours_Per_Day_xf (In  [(None, 1)]         0           []                               
 putLayer)                                                                                        
                                                                                                  
 BMI_xf (InputLayer)            [(None, 1)]          0           []                               
                                                                                                  
 Exercise_Hours_Per_Week_xf (In  [(None, 1)]         0           []                         

INFO:tensorflow:Assets written to: pipelines\hanhanhanny-pipeline\Trainer\model\7\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\hanhanhanny-pipeline\Trainer\model\7\Format-Serving\assets


100/100 [==============================] - 6s 43ms/step - loss: 0.6889 - binary_accuracy: 0.6084 - val_loss: 0.6472 - val_binary_accuracy: 0.6534
Epoch 2/10
 96/100 [===========================>..] - ETA: 0s - loss: 0.6588 - binary_accuracy: 0.6418
Epoch 2: val_binary_accuracy improved from 0.65343 to 0.65827, saving model to pipelines\hanhanhanny-pipeline\Trainer\model\7\Format-Serving


INFO:tensorflow:Assets written to: pipelines\hanhanhanny-pipeline\Trainer\model\7\Format-Serving\assets


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines\hanhanhanny-pipeline\Trainer\model\7\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\hanhanhanny-pipeline\Trainer\model\7\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

## Analisis dan Validasi Model

Komponen Resolver
Digunakan untuk membandingkan model baru dengan versi model sebelumnya yang menyediakan baseline model yang digunakan untuk pembanding.

In [22]:
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing

model_resolver = Resolver(
    strategy_class=LatestBlessedModelStrategy,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing)
).with_id("latest_blessed_model_resolver")

interactive_context.run(model_resolver)

ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

Komponen Evaluator

In [23]:
import tensorflow_model_analysis as tfma

eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='Heart Attack Risk')],
    slicing_specs = [tfma.SlicingSpec()],
    metrics_specs = [
        tfma.MetricsSpec(metrics=[

            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                              threshold=tfma.MetricThreshold(
                                  value_threshold=tfma.GenericValueThreshold(
                                      lower_bound={'value': 0.5}),
                                  change_threshold=tfma.GenericChangeThreshold(
                                      direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                                      absolute={'value': 0.0001})
                              )
            )
        ]),
        tfma.MetricsSpec(
            # Add confusion matrix
            metrics=[
                tfma.MetricConfig(
                    class_name='ConfusionMatrixPlot'
                )
            ],
        )
    ]
)

In [24]:
from tfx.components import Evaluator

evaluator = Evaluator(
    examples=example_gen.outputs["examples"],
    model=trainer.outputs["model"],
    baseline_model=model_resolver.outputs["model"],
    eval_config=eval_config
)

interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 9
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

In [25]:
# Visualize the evaluation metrics

# Load the evaluation result
tfma_result = tfma.load_eval_result(
    evaluator.outputs['evaluation'].get()[0].uri
)

# Render metrics and plot
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

## Model Deployment

Pusher menerima input dari trained model, hasil dari evaluasi dan file path dari model yang kemudian akan digunakan untuk proses development

In [26]:
from tfx.components import Pusher
from tfx.proto import pusher_pb2

pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory='serving_model_dir/heart-attack-risk-model'
        )
    )
)

interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 10
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))

In [28]:
import requests
from pprint import PrettyPrinter
    
pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8080/v1/models/heart-attack-risk-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1722270814'}]}
